In [20]:
path = 'C:/Users/82103/OneDrive/바탕 화면/산업분류AI공모전/'

import logging

def make_logger(name=None):
    #1 logger instance를 만든다.
    logger = logging.getLogger(name)

    #2 logger의 level을 가장 낮은 수준인 DEBUG로 설정해둔다.
    logger.setLevel(logging.DEBUG)

    #3 formatter 지정
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    #4 handler instance 생성
    console = logging.StreamHandler()
    file_handler = logging.FileHandler(filename=path + "logs/final_1.log",
                                       encoding = 'utf-8')
    
    #5 handler 별로 다른 level 설정
    console.setLevel(logging.INFO)
    file_handler.setLevel(logging.DEBUG)

    #6 handler 출력 format 지정
    console.setFormatter(formatter)
    file_handler.setFormatter(formatter)

    #7 logger에 handler 추가
    logger.addHandler(console)
    logger.addHandler(file_handler)

    return logger

In [21]:
logger = make_logger()

logger.debug("debug logging")
logger.info("info logging")
logger.warning("warning logging")
logger.error("error logging")
logger.critical("critical logging")

2022-04-12 23:35:17,666 - root - INFO - info logging
2022-04-12 23:35:17,669 - root - WARNING - warning logging
2022-04-12 23:35:17,670 - root - ERROR - error logging
2022-04-12 23:35:17,670 - root - CRITICAL - critical logging


In [ ]:
# !pip install git+https://github.com/ssut/py-hanspell.git

In [22]:
from hanspell import spell_checker
from tqdm import tqdm
import pandas as pd
import torch
torch.cuda.is_available()

True

In [23]:
data = pd.read_csv('./train.csv', encoding = 'cp949')
add_data = pd.read_csv('./add_index_fin_0411.csv', encoding = 'cp949')
code_data = pd.read_csv('./code_data.csv', encoding = 'cp949')

In [24]:
train_d = data.copy()
train_d = train_d.iloc[:,3:]

In [25]:
train_d.columns = ['y', 'x1', 'x2', 'x3']
cols = ['x1', 'x2', 'x3']

In [26]:
# na 값 빈칸으로 넣어서, nan 로 합쳐지는 거 방지
train_d = train_d.fillna("")
train_d['x'] = train_d[cols].astype('str').apply(' '.join, axis = 1)
train_d = train_d[['y','x']]

In [28]:
train_d = pd.concat([train_d, add_data,code_data], axis = 0, ignore_index=True)

In [29]:
train_d = train_d.drop_duplicates(keep='first').reset_index(drop=True) # 같은 항목들 삭제

In [30]:
# bert 모형의 경우 형태소 처리 없이, 특수문자만 제외한 문장만 input
# 문장의 흐름을 파악할 수 있기 때문
train_d['clean'] = train_d["x"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

In [31]:
train_d

,y,x,clean
0,952,카센터에서 자동차부분정비 타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,472,상점내에서 일반인을 대상으로 채소.과일판매,상점내에서 일반인을 대상으로 채소 과일판매
2,467,절단하여사업체에도매 공업용고무를가지고 합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,475,영업점에서 일반소비자에게 열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,872,어린이집 보호자의 위탁을 받아 취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육
...,...,...,...
705193,951,컴퓨터 및 통신장비 수리업,컴퓨터 및 통신장비 수리업
705194,952,자동차 및 모터사이클 수리업,자동차 및 모터사이클 수리업
705195,953,개인 및 가정용품 수리업,개인 및 가정용품 수리업
705196,961,"미용, 욕탕 및 유사 서비스업",미용 욕탕 및 유사 서비스업


In [32]:
# 맞춤법 교정 함수
# 참고 : https://hong-yp-ml-records.tistory.com/99

def comment_clean_t(data):
    comment = data['clean']
    comment_list = []
    for i in tqdm(range(len(comment))):    
        try:
            # 특정 특수문자 삭제
            sent = comment[i]

            hanspell_sent = spell_checker.check(sent).checked
            comment_list.append(hanspell_sent)
        except:
            comment_list.append(sent)
    return comment_list

In [33]:
clean_list = comment_clean_t(train_d)

100%|████████████████████████████████████████████████████████████████████████| 705198/705198 [5:01:26<00:00, 38.99it/s]


In [34]:
train_d['clean_done'] = clean_list

In [35]:
train_d.to_csv('./correct_train_fin.csv', index = False, encoding = "utf-8-sig")

In [36]:
# 제출파일도 맞춤법 검사
test_d = pd.read_table('./submission.txt', sep = '|')

In [37]:
test = test_d.copy()
cols = ['text_obj', 'text_mthd', 'text_deal']
# na 값 빈칸으로 넣어서, nan 로 합쳐지는 거 방지
test = test.fillna("")
test['x'] = test[cols].astype('str').apply(' '.join, axis = 1)
test['clean'] = test["x"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)

In [38]:
clean_list = comment_clean_t(test)

100%|██████████████████████████████████████████████████████████████████████████| 100000/100000 [42:58<00:00, 38.78it/s]


In [39]:
test['clean_done'] = clean_list

In [40]:
test.to_csv('./correct_sub_fin.csv', index = False, encoding = "utf-8-sig")